# **1️⃣ Installasi dan Import Library**

In [1]:
!pip install flask flask-cors pyngrok
!pip install gdown
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import conf, ngrok
import tensorflow as tf
import numpy as np
import pickle
import pandas as pd
import threading
import requests
import gdown
import logging
from datetime import datetime
import joblib
from sklearn.metrics.pairwise import cosine_similarity

## **2️⃣ Ngrok Authentication**

In [2]:
# Masukkan token kamu di sini
conf.get_default().auth_token = "2yGk1f8pQRPNaDUOuPuGfeyKWyD_54wGmdvqebnHBURhwFWGc"

## **3️⃣ Download Model dan Data**

In [3]:
gdown.download('https://drive.google.com/uc?id=1E0P3hRrOXw15e74gRRFzS-Lpy2lnDYWO', 'RekomendasibyLokasi_model.h5', quiet=False)
gdown.download('https://drive.google.com/uc?id=1NDjuhF3e7_dIeggp5DX2aoTUbqcONjUh', 'climate_encoder.pkl', quiet=False)
gdown.download('https://drive.google.com/uc?id=1GVu4BY5AjjsLqpdeEsWb9yhJV4qQNPty', 'label_encoder.pkl', quiet=False)
gdown.download('https://drive.google.com/uc?id=1mPVC3Gt6X3ahXXjsvoCMxMmNg_IX-V4-', 'RekomendasibyCustom_model.h5', quiet=False)
gdown.download('https://drive.google.com/uc?id=1NwaVTbBugi-ku8DDOQ_P8GHn6qD8Quq5', 'tfidf_vectorizer.pkl', quiet=False)
gdown.download('https://drive.google.com/uc?id=13LhafhE4-CwF6MKz2HdOkktkBo8ystYK', 'label_encoder_custom.pkl', quiet=False)
gdown.download('https://drive.google.com/uc?id=1_u-sI0bTmjO7hFblMx85HlJdvP3Ax_Os', 'encoder_houseplants.h5', quiet=False)
gdown.download('https://drive.google.com/uc?id=1e5AEgy9V9GjFL3UdXkxs1AdNnE56uCAz', 'encoder_plantsandfamily.h5', quiet=False)
gdown.download('https://drive.google.com/uc?id=1vaJq-sSny8TrpDlEmPoOppYmrwwpwyuY', 'plant_data.pkl', quiet=False)


Downloading...
From: https://drive.google.com/uc?id=1E0P3hRrOXw15e74gRRFzS-Lpy2lnDYWO
To: /content/RekomendasibyLokasi_model.h5
100%|██████████| 98.5k/98.5k [00:00<00:00, 36.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1NDjuhF3e7_dIeggp5DX2aoTUbqcONjUh
To: /content/climate_encoder.pkl
100%|██████████| 131/131 [00:00<00:00, 362kB/s]
Downloading...
From: https://drive.google.com/uc?id=1GVu4BY5AjjsLqpdeEsWb9yhJV4qQNPty
To: /content/label_encoder.pkl
100%|██████████| 3.84k/3.84k [00:00<00:00, 10.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1mPVC3Gt6X3ahXXjsvoCMxMmNg_IX-V4-
To: /content/RekomendasibyCustom_model.h5
100%|██████████| 485k/485k [00:00<00:00, 109MB/s]
Downloading...
From: https://drive.google.com/uc?id=1NwaVTbBugi-ku8DDOQ_P8GHn6qD8Quq5
To: /content/tfidf_vectorizer.pkl
100%|██████████| 1.37k/1.37k [00:00<00:00, 3.40MB/s]
Downloading...
From: https://drive.google.com/uc?id=13LhafhE4-CwF6MKz2HdOkktkBo8ystYK
To: /content/label_encoder_custom.pkl
100%|██

'plant_data.pkl'

## **4️⃣ Load Model dan Encoder**

In [4]:
# lokasi
model_lokasi = tf.keras.models.load_model('RekomendasibyLokasi_model.h5')

with open("climate_encoder.pkl", "rb") as f:
    climate_encoder = pickle.load(f)

with open("label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

recommendation_history_location = set()

# custom
model_custom = tf.keras.models.load_model('RekomendasibyCustom_model.h5')

with open("tfidf_vectorizer.pkl", "rb") as f:
    tfidf = pickle.load(f)

with open("label_encoder_custom.pkl", "rb") as f:
    label_encoder_custom = pickle.load(f)

recommendation_history_custom = set()

# kebun
encoder_paf = tf.keras.models.load_model("encoder_plantsandfamily.h5")
encoder_hpl = tf.keras.models.load_model("encoder_houseplants.h5")

## **5️⃣ Inisialisasi Flask App dan  Endpoint Home**

In [5]:
# Setup Flask App
app = Flask(__name__)
CORS(app)

# Endpoint Home
@app.route("/")
def home():
    return "🌱 Welcome to ML API!"


## **6️⃣ Endpoint Lokasi**

In [6]:
@app.route("/predict/lokasi", methods=["GET", "POST"])
def predict_lokasi():
    global recommendation_history_location
    if request.method == "GET":
        return jsonify({
            "message": "Gunakan metode POST dengan JSON body yang berisi 'climate' dan 'temperature'.",
            "example": {
                "climate": "Subtropical",
                "temperature": 20,
                "reset": False
            }
        }), 200
    try:
        data = request.json
        climate = data.get("climate")
        temp = data.get("temperature")
        reset = data.get("reset", False)

        if reset:
            recommendation_history_location = set()

        if not climate or climate not in climate_encoder:
            return jsonify({"error": "Iklim tidak dikenali"}), 400

        temperature = float(temp)
        climate_code = climate_encoder[climate]
        input_array = np.array([[climate_code, temperature]])
        preds = model_lokasi.predict(input_array, verbose=0)[0]

        labels = label_encoder.inverse_transform(np.arange(len(preds)))
        prob_df = pd.DataFrame({"latin": labels, "prob": preds})
        prob_df = prob_df[~prob_df["latin"].isin(recommendation_history_location)]

        top_latin = prob_df.sort_values(by="prob", ascending=False).head(3)["latin"].tolist()
        recommendation_history_location.update(top_latin)

        return jsonify({
            "input": {"climate": climate, "temperature": temperature},
            "recommendations": top_latin
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 400


## **7️⃣ Endpoint Custom**

In [7]:
@app.route("/predict/custom", methods=["GET", "POST"])
def predict_custom():
    global recommendation_history_custom
    if request.method == "GET":
        return jsonify({
            "message": "Gunakan POST dengan filter tanaman.",
            "required_filters": ["Growth", "Soil", "Sunlight", "Watering", "Fertilization Type"],
            "example": {
                "Growth": "none",
                "Soil": "well-drained",
                "Sunlight": "full sunlight",
                "Watering": "Keep soil evenly moist",
                "Fertilization Type": "Organic",
                "reset": False
            }
        }), 200
    try:
        data = request.json
        reset = data.get("reset", False)

        if reset:
            recommendation_history_custom = set()

        filter_keys = ["Growth", "Soil", "Sunlight", "Watering", "Fertilization Type"]
        if not all(k in data for k in filter_keys):
            return jsonify({"error": "Filter tidak lengkap"}), 400

        user_filters = {k: v for k, v in data.items() if k in filter_keys and v.lower() != "none"}

        if not user_filters:
            return jsonify({"error": "Tidak ada filter yang dipilih"}), 400

        user_query = ' '.join(user_filters.values())
        user_vector = tfidf.transform([user_query]).toarray()
        probs = model_custom.predict(user_vector, verbose=0)[0]
        top_indices = probs.argsort()[::-1]

        top_plants = []
        for idx in top_indices:
            name = label_encoder_custom.inverse_transform([idx])[0]
            if name not in recommendation_history_custom:
                top_plants.append(name)
                recommendation_history_custom.add(name)
            if len(top_plants) == 3:
                break

        return jsonify({
            "input": user_filters,
            "recommendations": top_plants
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 400


## **8️⃣ Load Data Embedding dan Plant Info**

In [8]:
data = joblib.load('plant_data.pkl')
plantsandfamily = data['plantsandfamily']
houseplants = data['houseplants']
pencocokanplantsandfamily = data['pencocokanplantsandfamily']
pencocokanhouseplants = data['pencocokanhouseplants']
embed_paf = data['embed_paf']
embed_hpl = data['embed_hpl']


## **9️⃣ Endpoint Kebun**

In [9]:
@app.route('/predict/kebun', methods=['GET', 'POST'])
def rekomendasi_perawatan():
    if request.method == 'GET':
        return jsonify({
            'description': 'API ini memberikan saran perawatan tanaman berdasarkan nama tanaman.',
            'method': 'POST',
            'input_format': {
                'nama_tanaman': 'string, contoh: "Aloe Vera"'
            },
            'output_format': {
                'Jenis Cahaya': 'string',
                'Pola Penyiraman': 'string',
                'Tipe Pupuk': 'string',
                'Serangga': 'string',
                'Fungsi Tanaman': 'string'
            },
            'note': 'Gunakan POST untuk mendapatkan rekomendasi perawatan.'
        })

    # POST request
    req_data = request.get_json()
    nama_tanaman = req_data.get('nama_tanaman', '').strip().lower()

    # SCENARIO 1: Tanaman di PlantsAndFamily
    match_paf = plantsandfamily[plantsandfamily['Plant Name'].str.lower() == nama_tanaman]
    if not match_paf.empty:
        sunlight = match_paf['Sunlight'].values[0]
        watering = match_paf['Watering'].values[0]
        pupuk = match_paf['Fertilization Type'].values[0]

        entry = pencocokanplantsandfamily[pencocokanplantsandfamily['Plant Name'].str.lower() == nama_tanaman]
        if entry.empty:
            return jsonify({'error': 'Data numerik tidak ditemukan untuk tanaman ini.'})

        encoded_input = encoder_paf.predict(entry[['Sunlight', 'Watering']].values)
        similarities = cosine_similarity(encoded_input, embed_hpl)
        best_match_index = np.argmax(similarities)

        match_latin = pencocokanhouseplants.iloc[best_match_index]['latin']
        match_house = houseplants[houseplants['latin'] == match_latin]

        return jsonify({
            'Jenis Cahaya': sunlight,
            'Pola Penyiraman': watering,
            'Tipe Pupuk': pupuk,
            'Serangga': match_house['insects'].values[0],
            'Fungsi Tanaman': match_house['use'].values[0]
        })

    # SCENARIO 2: Tanaman di Houseplants
    match_hpl = houseplants[houseplants['latin'].str.lower() == nama_tanaman]
    if not match_hpl.empty:
        cahaya = match_hpl['ideallight'].values[0]
        siram = match_hpl['watering'].values[0]
        serangga = match_hpl['insects'].values[0]
        fungsi = match_hpl['use'].values[0]

        entry = pencocokanhouseplants[pencocokanhouseplants['latin'].str.lower() == nama_tanaman]
        if entry.empty:
            return jsonify({'error': 'Data numerik tidak ditemukan untuk tanaman ini.'})

        encoded_input = encoder_hpl.predict(entry[['ideallight', 'watering']].values)
        similarities = cosine_similarity(encoded_input, embed_paf)
        best_match_index = np.argmax(similarities)

        match_plant = pencocokanplantsandfamily.iloc[best_match_index]['Plant Name']
        pupuk = plantsandfamily[plantsandfamily['Plant Name'] == match_plant]['Fertilization Type'].values[0]

        return jsonify({
            'Jenis Cahaya': cahaya,
            'Pola Penyiraman': siram,
            'Tipe Pupuk': pupuk,
            'Serangga': serangga,
            'Fungsi Tanaman': fungsi
        })

    # SCENARIO 3: Tanaman tidak ditemukan
    return jsonify({'message': 'belum ada saran perawatan untuk tanaman ini'})


## **🔟 Jalankan Flask Server di Thread Terpisah**

In [10]:
def run_flask():
    app.run(port=5000, debug=False, use_reloader=False)

threading.Thread(target=run_flask).start()

## **Ini contoh penggunaan ngrok yang tidak menggunakan public url, melainkan dari domain akun ngrok**

In [11]:
public_url = ngrok.connect(addr=5000, domain="intimate-admittedly-kangaroo.ngrok-free.app")

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
